In [1]:
# Groupe H :
# Nom - numéro d'étudiant
# 
# NGUYEN Huu Khang - 21506865
# 
# NGUYEN Hoai Nam -
# 
# NGUYEN Tran Tuan Nam -
#
# TRAN Thi Tra My - 21511002
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#Load file
data = pd.read_csv('Dataset/claim_extraction_18_10_2019_annotated.csv', sep=',')

data2 = data.copy()

In [2]:
#Import outil sklearn
import sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

# Sickit learn met régulièrement à jour des versions et indique des futurs warnings
# Ces deux lignes permettent de ne pas les afficher
import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

#Import outil nltk pour traitement
import nltk
import unicodedata
import contractions
import inflect
#nltk.download('all')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer



In [19]:
#Pré-traitement choisis : 

#On supprime les colonnes qui contiennent beaucoup des valeurs null et qui ne servent pas pour notre analyse
data3 = data.copy()
data3 = data3.drop('claimReview_author', 1)
data3 = data3.drop('claimReview_author_name', 1)
data3 = data3.drop('creativeWork_author_sameAs', 1)
data3 = data3.drop('claimReview_author_url', 1)
data3 = data3.drop('claimReview_source', 1)
data3 = data3.drop('extra_entities_author', 1)
data3 = data3.drop('extra_entities_body', 1)
data3 = data3.drop('extra_entities_claimReview_claimReviewed', 1)
data3 = data3.drop('extra_entities_keywords', 1)
print(data3.shape)
display(data3.info())

######Stop words
#initialize stop words type
stop_words = stopwords.words('english')

######Pre traiter le données du texte
def pretraiter(text):   
    if(isinstance(text, str) and text ) :
        #Remove contractions
        text = contractions.fix(text)

        #remove URL
        text = re.sub(r"http\S+", "", text)
        
        #remove photo url
        text = re.sub(r"pic.\S+", "", text)

        # Tokenizing
        tokenizedText = word_tokenize(text)

        # Put all words in lowercase
        tokenizedText = [word.lower() for word in tokenizedText]

        # Delete ponctuations
        tokenizedText = [word for word in tokenizedText if word.isalpha()]

        #remove stop_word
        tokenizedText = [word for word in tokenizedText if not word in stop_words]

        # Converting numbers
        inflectEngine = inflect.engine()
        newWords = []
        for word in tokenizedText:
            if word.isdigit():
                newWords.append(inflectEngine.number_to_words(word))
            else:
                newWords.append(word)
        tokenizedText = newWords

        #Remove non-ASCII characters from list of tokenized words
        new_words = []
        for word in tokenizedText:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
        tokenizeText = new_words

        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        text = [lemmatizer.lemmatize(word, pos = 'v') for word in tokenizedText]

        # Turning back tokens into a string
        text = "".join([" " + i for i in tokenizedText]).strip()

        return text
    
    
##### Prétraiter alternateName
def pretraiter_alternatename (text) :
    if(isinstance(text, str) and text ) :
        # Tokenizing
        tokenizedText = word_tokenize(text)
        
        # Put all words in lowercase
        tokenizedText = [word.lower() for word in tokenizedText]
        
        if tokenizedText[0] == 'true' :
            return 1
        else :
            return 0
        
    else :
        return 0


(39218, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39218 entries, 0 to 39217
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  39218 non-null  int64  
 1   claimReview_claimReviewed   39216 non-null  object 
 2   claimReview_datePublished   37913 non-null  object 
 3   claimReview_url             39218 non-null  object 
 4   creativeWork_author_name    17481 non-null  object 
 5   creativeWork_datePublished  17547 non-null  object 
 6   extra_body                  38866 non-null  object 
 7   extra_refered_links         35393 non-null  object 
 8   extra_tags                  30239 non-null  object 
 9   extra_title                 38869 non-null  object 
 10  rating_alternateName        38964 non-null  object 
 11  rating_bestRating           3432 non-null   float64
 12  rating_ratingValue          3432 non-null   float64
 13  rating_worstRating 

None

In [21]:
#Exécution pretraitement
data3['extra_body'] = data['extra_body'].apply(lambda x: pretraiter(x))

###index = 0
##nbLigne = len(data3['extra_body'])
#while index < nbLigne : 
#    if(not isinstance(data3['extra_body'][index], str)) :
#        data3 = data3.drop(data3.index[index])
#    else :
#        data3['extra_body'].loc[index] = [pretraiter(data3['extra_body'][index])]
#    index = index + 1
#data3.reset_index(drop=True, inplace=True)


"\nindex = 0\nnbLigne = len(data3['extra_body'])\nwhile index < nbLigne : \n    if(not isinstance(data3['extra_body'][index], str)) :\n        data3 = data3.drop(data3.index[index])\n    else :\n        data3['extra_body'].loc[index] = [pretraiter(data3['extra_body'][index])]\n    index = index + 1\ndata3.reset_index(drop=True, inplace=True)\n"

In [22]:
data3 = data3[data3['extra_body'].notna()]

In [26]:
data3['rating_alternateName'] = data['rating_alternateName'].apply(lambda x: pretraiter_alternatename(x))

In [33]:
X = data3['extra_body'].sample(3000)
y = data3['rating_alternateName'].sample(3000)

# Vectorisation avec TF-IDF

# In[ ]:

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X)

X = vectors.toarray()


validation_size=0.3 #30% du jeu de données pour le test
testsize= 1-validation_size
seed=30
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=validation_size,random_state=seed,test_size=testsize)

clf = GaussianNB()
clf.fit(X_train, y_train)
result = clf.predict(X_test)
print('\n accuracy :', accuracy_score(result, y_test),'\n')



 accuracy : 0.7528571428571429 

